In [1]:
# ======================================================
# STEP 4: 정책 매트릭스 × LR/RF × CBM 병합 (PID 기반)
# ======================================================

import os
import pandas as pd

print("=" * 70)
print("🚀 STEP 4: 정책 × LR/RF × CBM 병합 START")
print("=" * 70)

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------

OUT_DIR = "../results/final"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_PATH = f"{OUT_DIR}/merged_policy_analysis.csv"

POLICY_PATH = "../results/oxju/policy_matrix_final.csv"
LRRF_PATH   = "../results/hyun/employment_predictions.csv"
CBM_PATH    = "../results/RYUI/cbm_predictions.csv"

# ------------------------------------------------------
# 2️⃣ 파일 존재 확인
# ------------------------------------------------------

for p in [POLICY_PATH, LRRF_PATH, CBM_PATH]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"❌ 파일 없음: {p}")

print("✅ 입력 파일 확인 완료")

# ------------------------------------------------------
# 3️⃣ 데이터 로드 + pid 생성
# ------------------------------------------------------

df_policy = pd.read_csv(POLICY_PATH).reset_index(drop=True)
df_lrrf   = pd.read_csv(LRRF_PATH).reset_index(drop=True)
df_cbm    = pd.read_csv(CBM_PATH).reset_index(drop=True)

# 🔑 pid = row_id (전부 동일 규칙)
df_policy["pid"] = df_policy.index
df_lrrf["pid"]   = df_lrrf.index
df_cbm["pid"]    = df_cbm.index

print(f"📊 policy shape : {df_policy.shape}")
print(f"📊 lrrf shape   : {df_lrrf.shape}")
print(f"📊 cbm shape    : {df_cbm.shape}")

# ------------------------------------------------------
# 4️⃣ 병합 (pid 기준)
# ------------------------------------------------------

df_merged = (
    df_policy
      .merge(df_lrrf, on="pid", how="left", suffixes=("", "_lrrf"))
      .merge(df_cbm,  on="pid", how="left")
)

print("✅ 병합 완료")
print(f"📊 merged shape: {df_merged.shape}")

# ------------------------------------------------------
# 5️⃣ 병합 성공률 점검
# ------------------------------------------------------

if "cbm_p_success" in df_merged.columns:
    match_rate = df_merged["cbm_p_success"].notna().mean() * 100
    print(f"🔍 CBM 매칭 성공률: {match_rate:.2f}%")
else:
    print("⚠️ cbm_p_success 컬럼 없음")

# ------------------------------------------------------
# 6️⃣ CBM 개입 플래그 (선택)
# ------------------------------------------------------

if "지원필요도점수" in df_merged.columns and "cbm_pred" in df_merged.columns:
    df_merged["cbm_need_recourse"] = (
        (df_merged["cbm_pred"] == 0) &
        (df_merged["지원필요도점수"] >= 3)
    )

# ------------------------------------------------------
# 7️⃣ 저장
# ------------------------------------------------------

df_merged.to_csv(
    OUT_PATH,
    index=False,
    encoding="utf-8-sig"
)

print("💾 저장 완료")
print(f"📁 결과 파일: {OUT_PATH}")

print("=" * 70)
print("🎉 STEP 4 COMPLETE")
print("=" * 70)


🚀 STEP 4: 정책 × LR/RF × CBM 병합 START
✅ 입력 파일 확인 완료
📊 policy shape : (7749, 41)
📊 lrrf shape   : (61326, 11)
📊 cbm shape    : (11682, 3)
✅ 병합 완료
📊 merged shape: (7749, 53)
🔍 CBM 매칭 성공률: 100.00%
💾 저장 완료
📁 결과 파일: ../results/final/merged_policy_analysis.csv
🎉 STEP 4 COMPLETE
